In [1]:
import os
import json
import pickle

from collections import Counter

import pandas as pd
import numpy as np

# Setup

In [2]:
data_dir = os.path.join(os.curdir, "data")

raw_dataset_url = "https://nlp-slides.vercel.app/clean-tweets.tsv"

In [3]:
raw = pd.read_csv(filepath_or_buffer=raw_dataset_url, sep="\t")

raw.head()

,Tweet,Country,Topic,Sentiment,Sentiment_Expression,Sentiment_Target,word_count,char_count,clean_text,clean_stemmed
0,"""أنا أؤمن بأن الانسان ينطفئ جماله عند ابتعاد م...",lebanon,personal,negative,implicit,بريق العيون,23,132,اومن بان الانسان ينطفي جماله ابتعاد يحب بريق ا...,اوم بان انس نطف جمل بعد يحب برق عين خفي صبح ذب...
1,من الذاكره... @3FInQe . عندما اعتقد كريستيانو ...,jordan,sports,positive,explicit,افضل لاعب في العالم,23,141,الذاكره عندما اعتقد كريستيانو انه افضل لاعب ال...,ذكر عند عقد كريستيانو انه فضل لعب علم ككا يسي ...
2,لا نخلو من ضغوطات الحياة. فنحن نعيش على أرض أع...,palestine,personal,neutral,none,none,24,133,نخلو ضغوطات الحياه فنحن نعيش ارض اعدت للبلاء و...,خلو ضغط حيه فنح نعش ارض اعد بلء ولم سلم بيء وك...
3,#مصطلحات_لبنانيه_حيرت_البشريه بتوصل عالبيت ، ب...,lebanon,personal,negative,explicit,مصطلحات_لبنانيه,23,135,بتوصل عالبيت بنط بقلك جيت بتقعد لتتحدث معو بقل...,وصل علب بنط بقل جيت قعد حدث معو بقل شو تقم تمش...
4,نصمت !! لتسير حياتنا على مً يرام فالناّس لم تع...,palestine,personal,negative,explicit,س لم تعد كما ك,16,67,نصمت لتسير حياتنا يرام فالناس تعد كانت نقيه,نصم تسر حيت يرم لنس تعد كانت نقه


In [4]:
tweets = raw["clean_text"]

# Tokenization

First step is to split the dataset into small bits, each bit is called a `token`

once the corpus is tokenized, we can assign each unique `token` an `index`, note that the index value for any token is not important 

In [5]:
sample = tweets.iloc[0]
sample

'اومن بان الانسان ينطفي جماله ابتعاد يحب بريق العيون يختفي فيصبح ذابلا منطفيا يتحول ربيعه خريف'

Tokenizing have many forms, examples: 
- Word Level Tokenization
- Character Level Tokenization
- Sub-word Tokenization
- Byte Pair Encoding

In this chapter we will cover word level and character level tokenization, others will be covered in later chapters

In [6]:
# Implement a generic Tokenizer class
# Different tokenizers will inherit from this class
class Tokenizer:
    def __init__(self, corpus: list[str], min_frequency: int = None):
        self.min_frequency = min_frequency
        self.vocab = self._create_vocab(corpus=corpus)
        
        
    def _create_vocab(self, corpus: list[str]) -> dict[str, int]:
        ...
    
    def _tokenize_document(self, document: str) -> list[int]:
        ...
    
    def tokenize(self, documents: list[str]) -> list[list[int]]:
        return [self._tokenize_document(document) for document in documents]
    
    def __len__(self):
        return self.vocab

## Word Level

Here we split the sentence into indivual words, omitting the whitespaces between them

In [7]:
sample_tokens = sample.split(" ")
sample_tokens

['اومن',
 'بان',
 'الانسان',
 'ينطفي',
 'جماله',
 'ابتعاد',
 'يحب',
 'بريق',
 'العيون',
 'يختفي',
 'فيصبح',
 'ذابلا',
 'منطفيا',
 'يتحول',
 'ربيعه',
 'خريف']

In [8]:
# Create a list of all unique tokens in the corpus
all_word_tokens = [token for sample in tweets for token in sample.split(" ")]

word_level_tokens = set(all_word_tokens)
print(len(word_level_tokens), len(all_word_tokens))

19644 62252


In [9]:
word2idx = {token: index for index, token in enumerate(word_level_tokens)}
word2idx

{'': 0,
 'انتحاريات': 1,
 'نتمني': 2,
 'بجنن': 3,
 'تقاطع': 4,
 'وصفت': 5,
 'حال': 6,
 'رونالدو': 7,
 'النقل': 8,
 'وكما': 9,
 'اعزموا': 10,
 'نزل': 11,
 'موفقا': 12,
 'يظلل': 13,
 'ويكلمهم': 14,
 'بالاخطاء': 15,
 'مخطوف': 16,
 'اتفرج': 17,
 'بعصر': 18,
 'الانديه': 19,
 'خلص': 20,
 'واحرسهم': 21,
 'غايه': 22,
 'المنصه': 23,
 'داخل': 24,
 'تجيه': 25,
 'المذهب': 26,
 'خاصم': 27,
 'عندما': 28,
 'ويمتنع': 29,
 'والادب': 30,
 'در': 31,
 'مليارات': 32,
 'لبواب': 33,
 'بهتم': 34,
 'كومبارس': 35,
 'المقدمه': 36,
 'للبلاد': 37,
 'كتبهم': 38,
 'وتضيع': 39,
 'رضا': 40,
 'وحاطين': 41,
 'غطاك': 42,
 'ودراج': 43,
 'راموس': 44,
 'يشوف': 45,
 'العدوان': 46,
 'باذن': 47,
 'تجدد': 48,
 'وهاردلك': 49,
 'الخطرع': 50,
 'عبدا': 51,
 'انجازات': 52,
 'نفسيات': 53,
 'والفاسدون': 54,
 'تعليقا': 55,
 'صديقتي': 56,
 'يتاذي': 57,
 'امتحان': 58,
 'واشكره': 59,
 'بدويه': 60,
 'اختها': 61,
 'اسرقك': 62,
 'جليغم': 63,
 'مصدق': 64,
 'شله': 65,
 'التحقين': 66,
 'الصوره': 67,
 'مغربيه': 68,
 'التزوير': 69,
 'ليخبرك': 70,

In [10]:
class WordLevelTokenizer(Tokenizer):
    def __init__(self, corpus: list[str], min_frequency: int = 0):
        super().__init__(corpus=corpus, min_frequency=min_frequency)
        
    def _create_vocab(self, corpus: list[str]) -> dict[str, int]:
        tokens_counter = Counter([token for sample in corpus for token in sample.split(" ")])
        tokens = [token for token, count in tokens_counter.items() if count >= self.min_frequency]
        vocab = {token: index for index, token in enumerate(tokens, start=2)} 
        vocab["[PAD]"] = 0
        vocab["[OOV]"] = 1
        return vocab
    
    def _tokenize_document(self, document: str) -> list[int]:
        return [self.vocab.get(token, -1) for token in document.split(" ")]

    

In [12]:
word_level_tokenizer = WordLevelTokenizer(corpus=tweets, min_frequency=10)

In [13]:
word_level_tokenizer.vocab

{'بان': 2,
 'الانسان': 3,
 'عندما': 4,
 'اعتقد': 5,
 'انه': 6,
 'افضل': 7,
 'لاعب': 8,
 'العالم': 9,
 'حدث': 10,
 '😂': 11,
 'الحياه': 12,
 'نعيش': 13,
 'ارض': 14,
 'ولم': 15,
 'دايما': 16,
 'بالله': 17,
 'جيت': 18,
 'شو': 19,
 'شي': 20,
 '🤔': 21,
 'حياتنا': 22,
 'تعد': 23,
 'صاحب': 24,
 'السمو': 25,
 'الملكي': 26,
 'الامير': 27,
 'الدكتور': 28,
 'سعود': 29,
 'عبدالعزيز': 30,
 'امير': 31,
 'منطقه': 32,
 'مجلس': 33,
 'المنطقه': 34,
 'اكثر': 35,
 'خارج': 36,
 'علينا': 37,
 'الف': 38,
 'مبروك': 39,
 'للمنتخب': 40,
 'السوري': 41,
 'الجميع': 42,
 'لعب': 43,
 'شكرا': 44,
 'النهايه': 45,
 'الظروف': 46,
 'حلو': 47,
 'انو': 48,
 'بلا': 49,
 'احمد': 50,
 'رحمه': 51,
 'بال': 52,
 'كلام': 53,
 'قال': 54,
 'لانهم': 55,
 'الاسلام': 56,
 '🔴': 57,
 'شخص': 58,
 'دخل': 59,
 'قضيه': 60,
 'انتهي': 61,
 'المسوول': 62,
 'الدوله': 63,
 'الحكومه': 64,
 'الاردنيه': 65,
 'تعلن': 66,
 'اطلاق': 67,
 'مشروع': 68,
 'المدينه': 69,
 'الجديده': 70,
 'يكون': 71,
 'ترامب': 72,
 'سلمان': 73,
 'اي': 74,
 'وجود': 75,
 'حيات

In [14]:
len(word_level_tokenizer.vocab)

961

## Character Level Tokenization

In [15]:
sample_tokens = list(sample)
sample_tokens

['ا',
 'و',
 'م',
 'ن',
 ' ',
 'ب',
 'ا',
 'ن',
 ' ',
 'ا',
 'ل',
 'ا',
 'ن',
 'س',
 'ا',
 'ن',
 ' ',
 'ي',
 'ن',
 'ط',
 'ف',
 'ي',
 ' ',
 'ج',
 'م',
 'ا',
 'ل',
 'ه',
 ' ',
 'ا',
 'ب',
 'ت',
 'ع',
 'ا',
 'د',
 ' ',
 'ي',
 'ح',
 'ب',
 ' ',
 'ب',
 'ر',
 'ي',
 'ق',
 ' ',
 'ا',
 'ل',
 'ع',
 'ي',
 'و',
 'ن',
 ' ',
 'ي',
 'خ',
 'ت',
 'ف',
 'ي',
 ' ',
 'ف',
 'ي',
 'ص',
 'ب',
 'ح',
 ' ',
 'ذ',
 'ا',
 'ب',
 'ل',
 'ا',
 ' ',
 'م',
 'ن',
 'ط',
 'ف',
 'ي',
 'ا',
 ' ',
 'ي',
 'ت',
 'ح',
 'و',
 'ل',
 ' ',
 'ر',
 'ب',
 'ي',
 'ع',
 'ه',
 ' ',
 'خ',
 'ر',
 'ي',
 'ف']

In [35]:
char_level_tokens = [token for tweet in tweets for token in tweet]

char_level_tokens = set(char_level_tokens)
print(len(char_level_tokens))

325


In [36]:
char2idx = {token: index for index, token in enumerate(char_level_tokens)}
char2idx

{'💖': 0,
 'ﻭ': 1,
 '🙃': 2,
 'ص': 3,
 '🌈': 4,
 'ﻜ': 5,
 'ﻐ': 6,
 '🎼': 7,
 '👨': 8,
 'ش': 9,
 '💪': 10,
 '🕋': 11,
 'ق': 12,
 '🌺': 13,
 '♂': 14,
 '🤥': 15,
 '🚶': 16,
 '\u200c': 17,
 '🖕': 18,
 'گ': 19,
 '😐': 20,
 'س': 21,
 '👍': 22,
 '🇱': 23,
 '⌛': 24,
 '🙏': 25,
 'า': 26,
 '😤': 27,
 'ظ': 28,
 '🕊': 29,
 '💀': 30,
 'ﻖ': 31,
 'ﻌ': 32,
 '🍂': 33,
 '🙄': 34,
 '🍀': 35,
 '🦅': 36,
 '⚽': 37,
 '👌': 38,
 'ط': 39,
 '’': 40,
 '💛': 41,
 'ﺎ': 42,
 'ك': 43,
 '🇮': 44,
 'ﺣ': 45,
 'ﺤ': 46,
 '่': 47,
 '👑': 48,
 '🔺': 49,
 '🏴': 50,
 '🚦': 51,
 'ه': 52,
 '🇦': 53,
 '😉': 54,
 'ھ': 55,
 '📍': 56,
 '🔴': 57,
 '😔': 58,
 '😭': 59,
 '״': 60,
 '✊': 61,
 '💐': 62,
 '퍼': 63,
 'ح': 64,
 'ف': 65,
 '🐐': 66,
 '😢': 67,
 'ﻫ': 68,
 'چ': 69,
 '💜': 70,
 'ﻢ': 71,
 '👆': 72,
 '\u200d': 73,
 '😞': 74,
 '✔': 75,
 'ل': 76,
 '🍃': 77,
 '💤': 78,
 'ﻛ': 79,
 '🌝': 80,
 '🤣': 81,
 'ﺒ': 82,
 '🌱': 83,
 'خ': 84,
 '🙈': 85,
 '💎': 86,
 '☺': 87,
 '😱': 88,
 '😎': 89,
 '‼': 90,
 '📺': 91,
 '٪': 92,
 '💕': 93,
 '￼': 94,
 '↷': 95,
 '\u2069': 96,
 '😏': 97,
 'م': 98,
 'ﻬ':

In [37]:
# TODO: Implement character level tokenizer
# 1. __init__()
# 2. _create_vocab
# 3. _tokenize_document 
class CharacterLevelTokenizer(Tokenizer):
    ...

> Notice the difference in the vocabulary size, between word level and character level. Why would you choose one over the other? 

## Result of Tokenization

1. List of documents (corpus)
2. Each document is represented by a sequence of tokens

> Not all documents have the same length

In [16]:
tokenized_tweets = word_level_tokenizer.tokenize(tweets)

In [19]:
max([len(t) for t in tokenized_tweets])

82

# n-grams

To be continued

# Save the Tokenizer

One of the most straight forward ways of saving a Python object is through binary `serialization`

serialization is a method of converting the `object` to `bytes`, these `bytes` can be read later to recreate the object

`pickle` package is the built-in package for object serialization

In [20]:
word_level_tokenizer_path = os.path.join(data_dir, "word-tokenizer.pkl")
word_level_vocab_path = os.path.join(data_dir, "word-level-vocab.json")

with open(word_level_tokenizer_path, "wb+") as f:
    pickle.dump(obj=word_level_tokenizer, file=f)

    
with open(word_level_vocab_path, "wt+") as f:
    json.dump(word_level_tokenizer.vocab, f)

In [41]:
# TODO: Pickle character level tokenizer